<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/CoinGecko_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pycoingecko

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Available Coins { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

#General Information table

def coins_stats(url):
    """
    Generates a list of the coins on CoinGecko
    Args: url of the CoinGecko website

    """
    frames = []
    response = requests.get(url)
    data = response.json()

    
    token_list = pd.DataFrame(data)

    print('Stats: ')
    print(token_list.info(verbose=True))
    print(token_list.isnull().sum())
    print(token_list.describe())
   
    return token_list 


def main():
  url = "https://api.coingecko.com/api/v3/coins/list"

  coins_stats(url)
  

if __name__ == "__main__":
    main()

Stats: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13490 entries, 0 to 13489
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      13490 non-null  object
 1   symbol  13490 non-null  object
 2   name    13490 non-null  object
dtypes: object(3)
memory usage: 316.3+ KB
None
id        0
symbol    0
name      0
dtype: int64
            id symbol    name
count    13490  13490   13490
unique   13490  11216   13490
top     01coin   meta  01coin
freq         1     10       1


In [243]:
#@title  Download Coins Data Reports { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

 #suppressing carriers
id = token_list.id[:5]
currency = 'usd'
Horizon = 'max'

#loop  prices extraction to token list
try:

  start = 0
  ends =   2000

  for i in id[start : ends]:

    response = requests.get("https://api.coingecko.com/api/v3/coins/{}/market_chart?vs_currency={}&days={}".format(i,currency,Horizon))
    output = response.text
    json_parsed = json.loads(output)
    print(json_parsed)

    #Convert data to list of dataframes
    full_table = pd.json_normalize(json_parsed)       
    full_table['id'] = i
    #full_table.loc(i)
    frames.append(full_table)

    #Slow down requestsloc

    time.sleep(60/50)
    #Status report
    logging.info('Page '+ str(i) + ' – Rows ' + ' – source: '+ str(url))
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop
df = single

print(single)
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

single.to_excel(path + 'CoinGecko_prices_{}-{}.xlsx'.format(start,ends))

len(single)

,prices,market_caps,total_volumes,id
0,"[[1536969600000, 0.009788403063705275], [15370...","[[1536969600000, 30917.33141717348], [15370560...","[[1536969600000, 103.39712842361581], [1537056...",01coin
0,"[[1597708800000, 11007.434350568625], [1597795...","[[1597708800000, 0.0], [1597795200000, 0.0], [...","[[1597708800000, 61.06139804686967], [15977952...",0-5x-long-algorand-token
0,"[[1603324800000, 7786.404193192712], [16034112...","[[1603324800000, 0.0], [1603411200000, 0.0], [...","[[1603324800000, 49.75512279450143], [16034112...",0-5x-long-altcoin-index-token
0,"[[1604448000000, 6605.008367058888], [16045344...","[[1604448000000, 0.0], [1604534400000, 0.0], [...","[[1604448000000, 19.77103169697927], [16045344...",0-5x-long-ascendex-token-token
0,"[[1596672000000, 7240.231697506281], [15967584...","[[1596672000000, 0.0], [1596758400000, 0.0], [...","[[1596672000000, 1452.4628808367352], [1596758...",0-5x-long-bitcoin-cash-token


In [60]:
import itertools
n
print(list(itertools.chain(*df['prices'])))


for i in single['prices']:
  print(pd.json_normalize(single['prices'][i]))

[[1536969600000, 0.009788403063705275], [1537056000000, 0.009882558743828542], [1537142400000, 0.012145158274011318], [1537228800000, 0.007965204513941592], [1537315200000, 0.011866053006407695], [1537401600000, 0.011457442133231787], [1537488000000, 0.009856303171318184], [1537574400000, 0.010346361715446901], [1537660800000, 0.012509513238031308], [1537747200000, 0.00872956465326417], [1537833600000, 0.01005844860946907], [1537920000000, 0.008786264540711866], [1538006400000, 0.008104729360386247], [1538092800000, 0.00929708644129806], [1538179200000, 0.008332842978249768], [1538265600000, 0.00710890526630388], [1538352000000, 0.00704758470712929], [1538438400000, 0.010964302136757384], [1538524800000, 0.00783325848098873], [1538611200000, 0.00727669373334877], [1538697600000, 0.007948086657450218], [1538784000000, 0.010100972410891288], [1538870400000, 0.012831612472876296], [1538956800000, 0.011294693135382498], [1539043200000, 0.013286846597662993], [1539129600000, 0.0128280103037

In [297]:
import numpy as np

flat_list = []

for rows in single['id']:
  for i in single['prices']:  

    df = pd.DataFrame(i)
    df['id'] = rows
df



,0,1,id
0,1596672000000,7240.231698,0-5x-long-bitcoin-cash-token
1,1596758400000,7240.231698,0-5x-long-bitcoin-cash-token
2,1596844800000,6976.395764,0-5x-long-bitcoin-cash-token
3,1596931200000,7106.512949,0-5x-long-bitcoin-cash-token
4,1597017600000,7106.512949,0-5x-long-bitcoin-cash-token
...,...,...,...
233,1648252800000,8780.347729,0-5x-long-bitcoin-cash-token
234,1648339200000,8780.347729,0-5x-long-bitcoin-cash-token
235,1648425600000,9070.274067,0-5x-long-bitcoin-cash-token
236,1648503744000,9070.274067,0-5x-long-bitcoin-cash-token


In [57]:
df['prices.date'], df['prices.price'] = zip(*df.prices)